#### Reference

In [7]:
CARDS = {
    0: 'Sashimi',
    1: 'Egg Nigiri',
    2: 'Salmon Nigiri',
    3: 'Squid Nigiri',
    4: 'Wasabi',
    5: 'Tempura',
    6: 'Dumpling',
    7: '1 Maki',
    8: '2 Maki',
    9: '3 Maki',
    10: 'Pudding',  # Not implemented
    11: 'Chopsticks',  # Not implemented
}

### 4-Player Games

In [8]:
import pandas as pd
from State import *
from Player import *
import plotly.express as px
from tqdm import tqdm


state = State(get_actual_card_pool())
p1 = DeepPlayer('Deep Learning', num_of_opponents=3)
p2 = QPlayer('Q Learning')
p3 = RulePlayer('Rule Based')
p4 = DeepPlayer('Deep w/ Memory', num_of_opponents=3, memory_turns=3)
state.add_player(p1)
state.add_player(p2)
state.add_player(p3)
state.add_player(p4)
state.starting_hand_size = 8
df = pd.DataFrame()
all_results = []
hit_rates = []
        
for i in tqdm(range(10)):
    # Train
    p1.exp_rate = 0.3
    p2.exp_rate = 0.3
    p4.exp_rate = 0.3
    state.play_games(200, round_per_game=3)

    # Eval
    p1.exp_rate = 0
    p2.exp_rate = 0
    p4.exp_rate = 0
    state.stats = []
    state.play_games(200, round_per_game=3)
    all_results.append(state.stats)

100%|██████████| 10/10 [01:52<00:00, 11.21s/it]


In [9]:
df = pd.DataFrame()
for i in range(4):
    df[state.players[i].name] = [_[i] for _ in all_results]
fig = px.line(df)
fig.update_layout(
    title='4 Player Games stats, 200 Games per Round',
    xaxis_title='Round',
    yaxis_title='Games Won',
    legend_title='Player'
)
fig.show()

In [10]:
starting_input = [0] * len(CARD_ON_BOARD) + [1] * len(CARDS) + [0] * (3 * len(CARD_ON_BOARD)) + [0] * (3 * len(CARDS))
output = p4.model(torch.FloatTensor(starting_input))
fig = px.bar(y=output.detach().numpy(), x=list(CARDS.values()))
fig.update_layout(
    title='Deep Learning Model, value estimation on first action',
    xaxis_title=' ',
    yaxis_title='Estimated Value',
)
fig.show()

In [11]:
starting_input = [0] * len(CARD_ON_BOARD) + [1] * len(CARDS) + [0] * (3 * len(CARD_ON_BOARD))
output = p1.model(torch.FloatTensor(starting_input))
fig = px.bar(y=output.detach().numpy(), x=list(CARDS.values()))
fig.update_layout(
    title='Deep Learning Model, value estimation on first action',
    xaxis_title=' ',
    yaxis_title='Estimated Value',
)
fig.show()